In [ ]:
!pip install segmentation-models-pytorch

import pandas as pd
from global_config import *
from dataset.ship_dataset import ShipDataset
from models.segformer import Segformer
from processing.augmentation import train_transform, val_transform

In [ ]:
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True

In [ ]:
df = pd.read_csv(TRAIN_LABEL_PATH)
COEF_DIF = int(len(df)* 0.3)
df.shape, COEF_DIF

In [ ]:

df = pd.read_csv(TRAIN_LABEL_PATH, nrows=COEF_DIF)

num_nan_sample = min([int(len(df) * 0.3)])
nan_samples = df[df["EncodedPixels"].isna()]
drop_subset = nan_samples.sample(n=num_nan_sample, random_state=42)
df = df.drop(drop_subset.index)
df = df.groupby("ImageId")["EncodedPixels"].apply(list).reset_index()

df

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = ShipDataset(train_df, TRAIN_SAMPLE_DIR, train_transform)
val_dataset = ShipDataset(val_df, TRAIN_SAMPLE_DIR, val_transform)

In [ ]:
model = Segformer()
model.train(train_dataset, val_dataset)